<a href="https://colab.research.google.com/github/Ry02024/24DXstreamlit/blob/main/App_DashBorad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UIのイメージ（streamlit）

1. 単にデータをアップロードして分析結果と予測案を出力する

##環境構築

In [1]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.2 MB/s eta 0:00:00


In [2]:
import streamlit as st
from pyngrok import ngrok

In [3]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [4]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

## テストApp

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd

# Streamlitアプリのメイン部分
def main():
    # ブラウザ上に「hello streamlit」を表示
    st.title('テストアプリ')
    st.write('hello streamlit')

if __name__ == '__main__':
    main()

Writing streamlit_app.py


## データを読み込んで表示

In [5]:
!pip install japanize-matplotlib -q
import japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [6]:
# import pandas as pd
# sales_df = pd.read_csv('sales_history.csv')
# item_df = pd.read_csv('item_categories.csv')
# category_df = pd.read_csv('category_names.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sales_history.csv'

In [ ]:
display(sales_df)
display(item_df)
display(category_df)

In [7]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# Updated function to process dates correctly and plot in monthly intervals
def plot_line_chart_monthly(sales_df, y_column, title):
    # Ensure the '日付' column is in datetime format
    sales_df['日付'] = pd.to_datetime(sales_df['日付'])

    # Aggregate data by month for readability
    sales_by_date = sales_df.set_index('日付').resample('M')[y_column].sum().reset_index()

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 6))  # Wider figure
    ax.plot(sales_by_date['日付'], sales_by_date[y_column])
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(y_column)

    # Set x-axis to show monthly ticks and format the dates
    ax.xaxis.set_major_locator(mdates.MonthLocator())  # Major ticks every month
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format as Year-Month
    plt.xticks(rotation=45)  # Rotate x-axis labels for readability

    return fig


In [8]:
# Plot with the updated function
# fig = plot_line_chart_monthly(sales_df, '商品価格', '売上金額')
# plt.show()

NameError: name 'sales_df' is not defined

# 実行ファイルの用意

In [9]:
%%writefile streamlit_app.py
# Save this file as `streamlit_app.py`
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

# Load and merge data
def load_and_merge_data(sales_file, item_file, category_file):
    # Load each dataset
    sales_df = pd.read_csv(sales_file)
    item_df = pd.read_csv(item_file)
    category_df = pd.read_csv(category_file)

    # Merge sales data with item categories
    sales_df = sales_df.merge(item_df, on='商品ID', how='left')

    # Merge with category names to get 商品カテゴリ名
    sales_df = sales_df.merge(category_df, on='商品カテゴリID', how='left')

    return sales_df

# Create pie chart for category sales distribution
def plot_pie_chart(sales_df):
    category_sales = sales_df.groupby('商品カテゴリ名')['売上個数'].sum()
    fig, ax = plt.subplots()
    ax.pie(category_sales, labels=None, autopct='%1.1f%%', startangle=90, pctdistance=0.85)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.legend(category_sales.index, loc="center left", bbox_to_anchor=(1, 0.5), fontsize='small')
    plt.title("売上カテゴリの割合")
    return fig

import matplotlib.dates as mdates

# Updated function to process dates correctly and plot in monthly intervals
def plot_line_chart(sales_df, y_column, title):
    # Ensure the '日付' column is in datetime format
    sales_df['日付'] = pd.to_datetime(sales_df['日付'])

    # Aggregate data by month for readability
    sales_by_date = sales_df.set_index('日付').resample('M')[y_column].sum().reset_index()

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 6))  # Wider figure
    ax.plot(sales_by_date['日付'], sales_by_date[y_column])
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(y_column)

    # Set x-axis to show monthly ticks and format the dates
    ax.xaxis.set_major_locator(mdates.MonthLocator())  # Major ticks every month
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format as Year-Month
    plt.xticks(rotation=45)  # Rotate x-axis labels for readability

    return fig


# Create bar chart for top products by sales amount
def plot_bar_chart(sales_df):
    top_products = sales_df.groupby(['商品ID', '商品カテゴリ名'])['商品価格'].sum().nlargest(10).reset_index()
    fig, ax = plt.subplots()
    ax.barh(top_products['商品カテゴリ名'], top_products['商品価格'])
    ax.set_xlabel("売上金額")
    ax.set_ylabel("商品カテゴリ名")
    return fig

# Streamlit app main function
def main():
    st.title('売上ダッシュボード')

    # File uploaders
    st.sidebar.title("データファイルのアップロード")
    sales_file = st.sidebar.file_uploader("売上データファイル (sales_history.csv)", type=["csv"])
    item_file = st.sidebar.file_uploader("商品データファイル (item_categories.csv)", type=["csv"])
    category_file = st.sidebar.file_uploader("カテゴリデータファイル (category_names.csv)", type=["csv"])

    # Load and process data if all files are uploaded
    if sales_file and item_file and category_file:
        sales_df = load_and_merge_data(sales_file, item_file, category_file)

        # Display KPIs
        total_sales_quantity = sales_df['売上個数'].sum()
        total_sales_amount = (sales_df['商品価格'] * sales_df['売上個数']).sum()
        st.metric(label="売上個数", value=f"{total_sales_quantity:,}")
        st.metric(label="売上金額", value=f"{total_sales_amount:,.0f}")

        # Pie Chart
        st.subheader("売上カテゴリの割合")
        st.pyplot(plot_pie_chart(sales_df))

        # Line Charts
        st.subheader("売上金額の推移")
        st.pyplot(plot_line_chart(sales_df, '商品価格', '売上金額'))
        st.subheader("売上個数の推移")
        st.pyplot(plot_line_chart(sales_df, '売上個数', '売上個数'))

        # Bar Chart
        st.subheader("商品別売上金額トップ")
        st.pyplot(plot_bar_chart(sales_df))

        # Data Table
        st.subheader("売上トップ10商品")
        top_products_df = sales_df.groupby(['商品ID', '商品カテゴリ名'])['商品価格'].sum().nlargest(10).reset_index()
        st.dataframe(top_products_df)
    else:
        st.warning("すべてのデータファイルをアップロードしてください。")

if __name__ == '__main__':
    main()


Writing streamlit_app.py


##アプリの起動

In [10]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://5e61-35-236-185-99.ngrok-free.app" -> "http://localhost:8501"


In [11]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

/content/streamlit_app.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sales_by_date = sales_df.set_index('日付').resample('M')[y_column].sum().reset_index()
/content/streamlit_app.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sales_by_date = sales_df.set_index('日付').resample('M')[y_column].sum().reset_index()
